## **CUDA STF 教程 - Part 3: 任务 (Tasks)**

在 CUDA STF 中，**任务 (Task)** 是计算的基本单元。它可以是一个 CUDA 核函数、一个主机端函数，甚至是调用一个库函数。STF 的强大之处在于它能够根据任务对逻辑数据的访问模式自动推断依赖关系，并构建一个高效的执行图。

本部分将主要依据您提供的文档中 "Tasks" 章节 (Page 12-16) 的内容。

### **5. 任务创建与数据依赖 (文档 Page 12-13)**

任务是通过上下文对象（例如 ctx 或在 scheduler.execute lambda 中的 stf_ctx）的 task() 成员函数（或其变体如 cuda_kernel()、host_launch() 等）创建的。

#### **5.1 基本任务创建语法**

一个典型的任务创建涉及以下几个方面：

1. **执行位置 (Execution Place)**: 可选参数，指定任务在哪里执行（例如，特定 GPU 或主机）。如果未提供，则默认为当前 CUDA 设备 (exec_place::current_device())。我们将在后续的 "Places" 部分详细讨论。  
2. **数据依赖列表 (Data Dependencies)**: 这是至关重要的一步。您需要列出此任务将访问的所有逻辑数据，并为每个逻辑数据指定其**访问模式 (access mode)**。  
   * logical_data_handle.read(): 只读访问。  
   * logical_data_handle.write(): 只写访问（会覆盖数据，不保证读取旧值）。  
   * logical_data_handle.rw(): 读写访问。  
   * logical_data_handle.reduce(reducer, [no_init_tag]): 归约访问（例如求和、最大值等）。我们将在 parallel_for 部分详细介绍。  
3. **任务体 (Task Body)**: 通常是一个 C++ lambda 函数，它定义了任务实际执行的工作。

**通用 ctx.task() 语法示例 (来自文档 Page 12):**

下面的代码演示了如何使用 `ctx.task()` 创建一个在设备上执行 AXPY 操作的任务。首先，我们将代码写入 `p3_01_axpy_ctx_task.cu` 文件。

In [1]:
%%writefile p3_01_axpy_ctx_task.cu
#include <cuda/experimental/stf.cuh>  
#include <vector>  
#include <cmath>  
#include <iostream>
#include <numeric> // For std::iota if needed, or remove if not

// 假设我们有这样一个 AXPY 核函数，它直接操作 slice  
template <typename T>  
__global__ void axpy_slice_kernel(T alpha, cuda::experimental::stf::slice<const T> x, cuda::experimental::stf::slice<T> y) {  
    int tid = blockIdx.x * blockDim.x + threadIdx.x;  
    int nthreads = gridDim.x * blockDim.x;  
    for (size_t ind = tid; ind < x.size(); ind += nthreads) {  
        y(ind) += alpha * x(ind);  
    }  
}

int main() {  
    using namespace cuda::experimental::stf; // 为简洁起见  
    context ctx; // 使用通用上下文

    const size_t N = 1024;  
    double alpha = 2.0;

    // 1. 创建主机数据  
    std::vector<double> h_x_vec(N);  
    std::vector<double> h_y_vec(N);  
    for(size_t i = 0; i < N; ++i) {  
        h_x_vec[i] = (double)i;  
        h_y_vec[i] = (double)(N - i);  
    }

    // 2. 创建逻辑数据 (STF将负责H2D拷贝)  
    auto lX = ctx.logical_data(h_x_vec.data(), h_x_vec.size());  
    auto lY = ctx.logical_data(h_y_vec.data(), h_y_vec.size());  
    lX.set_symbol("X_vec");  
    lY.set_symbol("Y_vec");

    // 3. 定义核函数启动配置  
    dim3 num_blocks( (N + 255) / 256 );  
    dim3 threads_per_block(256);

    // 4. 创建并提交任务  
    ctx.task(exec_place::current_device(), 
             lX.read(),      
             lY.rw()         
            )  
        ->*[&](cudaStream_t s,                         
               slice<const double> sX_kernel_arg,     
               slice<double>       sY_kernel_arg      
              ) {  
        std::cout << "AXPY Task Body: Submitting kernel to stream " << s << std::endl;  
        axpy_slice_kernel<<<num_blocks, threads_per_block, 0, s>>>(alpha, sX_kernel_arg, sY_kernel_arg);  
    }; 

    // 5. 等待所有任务完成  
    ctx.finalize();

    std::cout << "First element of Y after AXPY: " << h_y_vec[0] << std::endl;  
    double expected_y0 = alpha * 0.0 + (double)N;
    std::cout << "Expected: " << expected_y0 << std::endl;
    if (std::abs(h_y_vec[0] - expected_y0) < 1e-5) {
        std::cout << "Result is correct!" << std::endl;
    } else {
        std::cout << "Result is INCORRECT!" << std::endl;
    }

    return 0;  
}

Writing p3_01_axpy_ctx_task.cu


现在，我们编译并运行这个示例：
*(注意: 您可能需要根据您的 GPU 修改 `-arch=sm_86` 参数。常见值有 `sm_70`, `sm_75`, `sm_80`, `sm_86`, `sm_90` 等。)*

In [4]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include p3_01_axpy_ctx_task.cu -o p3_01_axpy_ctx_task -arch=sm_86 -lcuda
!./p3_01_axpy_ctx_task

AXPY Task Body: Submitting kernel to stream 0x555556038f60
First element of Y after AXPY: 1024
Expected: 1024
Result is correct!


**lambda 函数的参数:**

* 第一个参数通常是 `cudaStream_t stream` (或简写为 `s`)。这是 STF 为此任务提供的 CUDA 流。您应该将所有异步 CUDA 操作（如核函数启动、异步内存拷贝）提交到这个流中。  
* 后续参数对应于您在 `ctx.task(...)` 中声明的每个逻辑数据依赖。STF 会将相应逻辑数据在任务执行位置的有效**数据实例 (data instance)** 传递给 lambda。  
  * 如果逻辑数据以 `.read()` 模式访问，则对应的数据实例类型通常是 `slice<const T>` (或类似的基础类型)。  
  * 如果以 `.rw()` 或 `.write()` 模式访问，则是 `slice<T>`。  
  * 使用 `auto` 关键字可以简化类型声明：`auto sX, auto sY`。

**重要说明 (文档 Page 13):**  
任务构造的主体 (lambda 函数) 是在任务提交时立即在主机上执行的，而不是在任务实际准备好执行时（即其依赖满足时）。因此，任务主体的作用是将异步工作（例如 CUDA 核函数）提交到传递给它的 CUDA 流中，而不是它本身就是 CUDA 核函数。试图在 lambda 函数中立即直接使用传递进来的 slice 数据（例如在 CPU 上访问其内容）是错误的，除非这些数据明确位于主机并且您已确保同步。正确的方式是将这些 slice 作为参数传递给与该流同步的 CUDA 核函数。CUDA 的执行语义将确保当核函数实际运行时，这些 slice 是有效的。

#### **5.2 cuda_kernel 和 cuda_kernel_chain (回顾 Part 1)**

我们在 Part 1 的 `01-axpy.cu` 示例分析中已经接触过 `stf_ctx.cuda_kernel(...)`。这是一种更直接的方式来提交单个 CUDA 核函数任务。

* **`ctx.cuda_kernel(dependencies...) ->* [](){ return cuda_kernel_desc{...}; }`**:  
  * lambda 函数体**返回一个 `cuda_kernel_desc` 对象**。  
  * `cuda_kernel_desc` 包含了核函数指针、启动配置 (grid/block dims, shared memory) 和核函数参数。  
  * 这种方式对于 CUDA Graph 后端可能更高效，因为它避免了 `ctx.task()` 可能涉及的图捕获开销。  
* **`ctx.cuda_kernel_chain(dependencies...) ->* [](){ return std::vector<cuda_kernel_desc>{...}; }`**:  
  * 类似于 `cuda_kernel`，但 lambda 返回一个 `std::vector<cuda_kernel_desc>`。  
  * 允许在一个 STF 任务中按顺序执行多个 CUDA 核函数。

**请参考您本地的 [`stf/01-axpy-cuda_kernel.cu`](./stf/01-axpy-cuda_kernel.cu) 和 [`stf/01-axpy-cuda_kernel_chain.cu`](./stf/01-axpy-cuda_kernel_chain.cu) (如果存在) 或回顾 `01-axpy.cu` (Part 1中的版本) 中 `cuda_kernel` 的用法。**

In [ ]:
// 概念性回顾 stf/01-axpy.cu (或 Part 1 中的 axpy.cu) 中 cuda_kernel 的用法  
// scheduler sched; // Or context ctx;
// sched.execute([&](auto& stf_ctx) { // or directly use ctx 
//     stf_ctx.cuda_kernel(exec_place::current_device(),  
//                        ld_x.reads(),  
//                        ld_y.reads_writes())  
//         .set_symbol("axpy_task_direct_kernel")  
//         ->*[&]() { // 这个 lambda 返回一个 cuda_kernel_desc  
//             return cuda::experimental::stf::cuda_kernel_desc{  
//                 axpy_stf_kernel<double>, // Kernel function pointer 
//                 num_blocks,              // Grid dimensions 
//                 threads_per_block,       // Block dimensions 
//                 0,                       // Shared memory (bytes) 
//                 alpha,                   // Kernel argument 1 
//                 // STF 会自动从 ld_x, ld_y 获取设备上的 slice 实例  
//                 // 并作为后续参数传递给 axpy_stf_kernel  
//                 // 在 cuda_kernel_desc 中直接使用逻辑数据句柄即可  
//                 ld_x,                    // Kernel argument 2 (becomes slice<const double>)
//                 ld_y                     // Kernel argument 3 (becomes slice<double>)
//             };  
//         };  
// }); // or ctx.finalize();

#### **5.3 多任务与依赖推断 (Example of creating and using multiple tasks - 文档 Page 14-16)**

当您提交多个任务时，CUDASTF 会根据它们声明的数据依赖关系自动推断执行顺序，构建任务图。

**文档 Page 14 的示例逻辑 (伪代码)：**

In [ ]:
// 伪代码，说明依赖关系 (假设 K1, K2, K3, K4 是定义的核函数，callback是主机函数) 
// context ctx; 
// auto lX = ctx.logical_data(X_host_data);  
// auto lY = ctx.logical_data(Y_host_data);

// Task 1: 读取 lX 和 lY，执行 K1, K2  
// ctx.task(lX.read(), lY.read())->*[&](cudaStream_t s, auto sX, auto sY) {  
//     K1<<<..., s>>>(sX, sY);  
//     K2<<<..., s>>>(sX, sY);  
// };

// Task 2: 读写 lX，执行 K3  
// ctx.task(lX.rw())->*[&](cudaStream_t s, auto sX) {  
//     K3<<<..., s>>>(sX);  
// };

// Task 3: 读写 lY，执行 K4  
// ctx.task(lY.rw())->*[&](cudaStream_t s, auto sY) {  
//     K4<<<..., s>>>(sY);  
// };

// Task 4 (Host Task): 读取 lX 和 lY，执行 callback  
// ctx.host_launch(lX.read(), lY.read())->*[&](auto sX_host, auto sY_host) { // Params are host-side instances 
//     callback(sX_host, sY_host); 
// };

// ctx.finalize();

**依赖分析 (文档 Page 14):**

* **Task 2 和 Task 3 依赖于 Task 1**:  
  * Task 2 修改 (rw) lX，而 Task 1 读取 (read) lX (WAR 依赖)。  
  * Task 3 修改 (rw) lY，而 Task 1 读取 (read) lY (WAR 依赖)。  
  * 因此，Task 2 和 Task 3 必须在 Task 1 完成后执行。  
  * 由于 Task 2 和 Task 3 操作不同的逻辑数据 (lX vs lY)，它们之间没有直接依赖，**可以并发执行** (在 Task 1 完成后)。  
* **Task 4 依赖于 Task 2 和 Task 3**:  
  * Task 4 在主机上读取 (read) lX，而 lX 被 Task 2 修改 (rw) (RAW 依赖)。  
  * Task 4 在主机上读取 (read) lY，而 lY 被 Task 3 修改 (rw) (RAW 依赖)。  
  * 因此，Task 4 必须在 Task 2 和 Task 3 都完成后执行。

文档 Page 15 展示了由此产生的任务图，Page 16 进一步展示了包含自动数据分配和传输的更详细的图。这突出了 STF 如何减轻程序员管理复杂异步操作和数据移动的负担。

请参考您本地的 [`stf/01-axpy-cuda_kernel_chain.cu`](./stf/01-axpy-cuda_kernel_chain.cu)。  
这个示例应该演示了如何顺序执行多个（可能是两个）AXPY 操作，例如：

1. Z = a*X + Y  
2. W = b*Z + Q

观察它是如何定义多个 `cuda_kernel` (或 `cuda_kernel_chain` 中的多个 `cuda_kernel_desc`)，以及如何通过共享的逻辑数据 `lZ` 来建立它们之间的依赖关系。`lZ` 在第一个任务中是 `writes()` 或 `rw()`，在第二个任务中是 `reads()`。

#### **5.4 主机端任务 (Host-Side Task Execution with host_launch) (文档 Page 14, 19)**

除了在 GPU 上执行计算，STF 还允许您将主机 (CPU) 上的函数作为任务集成到图中。这是通过 `ctx.host_launch()` (或 `stf_ctx.host_task()`，取决于上下文对象) 实现的。

**语法 (文档 Page 19, 55):**

In [ ]:
// ctx.host_launch(logicalData1.accessMode(), logicalData2.accessMode()...)  
//     ->*[capture list] (auto data1_host_instance, auto data2_host_instance...) {  
//     // 主机任务的实现代码  
//     // data1_host_instance, data2_host_instance 是在主机上有效的对应数据的实例 
// };

* 与 GPU 任务类似，您需要声明对逻辑数据的访问模式。  
* lambda 函数的参数是相应逻辑数据在主机上的有效实例。  
* `host_launch` 的一个重要特性是，它通过将 lambda 函数作为 CUDA 回调来调用，从而保持了整个工作负载的最佳异步语义。这意味着 lambda 函数的执行会正确地排队，等待其 GPU 数据依赖项准备就绪（包括必要的设备到主机的数据传输，这些由 STF 自动处理）。  
* 这与文档 Page 19 早期提到的 `ctx.task(exec_place::host(), ...)` 有所不同。`ctx.task(exec_place::host(), ...)` 仍然会立即执行 lambda（该 lambda 接收一个 `cudaStream_t`），并且需要用户在该 lambda 内部显式处理与 CUDA 流的同步（例如 `cudaStreamSynchronize`），这在 `graph_ctx` 后端是不允许的。**因此，`ctx.host_launch` 是推荐的执行主机任务的方式，因为它与所有后端兼容。**

请参考您本地的 [`stf/02-axpy-host_launch.cu`](./stf/02-axpy-host_launch.cu)。  
分析这个示例：

1. 它可能首先在 GPU 上执行一个 AXPY 操作。  
2. 然后，它使用 `host_launch` 提交一个主机任务。  
3. 这个主机任务可能依赖于 GPU AXPY 的结果（例如，读取修改后的 `lY`）。观察 `lY` 是如何以 `read()` 模式传递给 `host_launch` 的。  
4. 主机任务 lambda 内部的代码会在 CPU 上执行，并且可以安全地访问 `lY` 的主机数据实例。

**示例片段 (概念性，请对照 [`stf/02-axpy-host_launch.cu`](./stf/02-axpy-host_launch.cu)):**

In [ ]:
// context ctx; // Or scheduler sched; and auto& stf_ctx = sched.get_context();
// ... (GPU AXPY 任务提交，修改了 lY) ...

// // 主机任务，读取 lY 的结果  
// ctx.host_launch(exec_place::host(), // 明确指定主机执行 (host_launch 隐含了主机执行, exec_place::host()是可选的但更清晰)  
//                     lY.read()           // 依赖于 lY 的最新值  
//                    )  
//     .set_symbol("verify_on_host")  
//     ->*[&](cuda::experimental::stf::slice<const double> sY_host_instance) { // 注意参数类型 
//         // 此代码在主机上执行，sY_host_instance 是 lY 在主机内存中的有效副本  
//         // (如果 lY 最初是 std::vector, sY_host_instance 常常可以直接访问其原始内存，具体取决于 STF 实现和数据类型) 
//         std::cout << "Host Task: Verifying Y[0] = " << sY_host_instance(0) << std::endl;  
//         // ... 进行验证 ...  
//     };
// ctx.finalize();

### **动手试试:**

1. **编译并运行 [`stf/01-axpy-cuda_kernel_chain.cu`](./stf/01-axpy-cuda_kernel_chain.cu) 和 [`stf/02-axpy-host_launch.cu`](./stf/02-axpy-host_launch.cu)。**
   *注意: 您可能需要根据您的 GPU 修改 `-arch=sm_86` 参数。*

编译并运行 `stf/01-axpy-cuda_kernel_chain.cu`:

In [5]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include ./stf/01-axpy-cuda_kernel_chain.cu -o p3_axpy_kernel_chain -arch=sm_86 -lcuda
!./p3_axpy_kernel_chain

编译并运行 `stf/02-axpy-host_launch.cu`:

In [6]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include ./stf/02-axpy-host_launch.cu -o p3_axpy_host_launch -arch=sm_86 -lcuda
!./p3_axpy_host_launch

2. 在 [`stf/01-axpy-cuda_kernel_chain.cu`](./stf/01-axpy-cuda_kernel_chain.cu) 中，尝试修改第二个 AXPY 操作，使其不依赖于第一个操作的结果（例如，让它操作一组全新的数据）。然后生成任务图，观察图结构是否反映了这种并行性（即两个任务是否可以独立执行）。
   运行以下命令 (在修改并重新编译 `p3_axpy_kernel_chain` 之后) 生成任务图 `p3_axpy_kernel_chain.png`：

In [11]:
!CUDASTF_DOT_FILE=p3_axpy_kernel_chain.dot ./p3_axpy_kernel_chain
!dot -Tpng p3_axpy_kernel_chain.dot -o p3_axpy_kernel_chain.png
#[ -f p3_axpy_kernel_chain.png ] && echo "Task graph p3_axpy_kernel_chain.png generated." || echo "Failed to generate p3_axpy_kernel_chain.png."

   您可以在文件浏览器中打开 `p3_axpy_kernel_chain.png` 查看，或者如果 Jupyter 环境支持，可以在下一个 Markdown 单元格中显示它：
   ```markdown
   ![Task Graph for Kernel Chain](p3_axpy_kernel_chain.png)
   ```
   ![Task Graph for Kernel Chain](p3_axpy_kernel_chain.png)

3. 在 [`stf/02-axpy-host_launch.cu`](./stf/02-axpy-host_launch.cu) 的主机任务 lambda 中添加一些计算或打印语句，以确认它确实在主机上执行，并且可以访问从 GPU 同步过来的数据 (重新编译并运行 `p3_axpy_host_launch` 进行验证)。

4. **思考 `ctx.task(exec_place::host(), ...)` 和 `ctx.host_launch(...)` 的区别。** 为什么 `host_launch` 通常是更推荐的方式？(提示：异步性，与 `graph_ctx` 后端的兼容性)。

我们已经深入探讨了 STF 中任务的定义、数据依赖的声明、多种任务提交方式 (`task`, `cuda_kernel`, `host_launch`) 以及 STF 如何自动推断任务间的依赖关系。

在 Part 4 中，我们将学习 **"Synchronization" (同步)**、**"Places" (位置)** 的概念，以及更高级的任务构造原语如 **`parallel_for`** 和 **`launch`**。